# PARTE 2: DATA UNDERSTANDING

## Importación de librerias

In [ ]:
import time

import warnings
warnings.simplefilter(action = 'ignore')

import os

import numpy as np
import pandas as pd

import sklearn
from sklearn import set_config

from sklearn.tree import DecisionTreeClassifier

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler, OrdinalEncoder, OneHotEncoder

from sklearn.pipeline import Pipeline, FunctionTransformer
from sklearn.compose import ColumnTransformer

from sklearn.metrics import accuracy_score

import xgboost

import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import mannwhitneyu
from scipy.stats import chi2_contingency
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
import matplotlib.pyplot as plt # Librería para la visualización de datos
import seaborn as sns # Librería para la visualización de datos
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, OrdinalEncoder # Librería para crear modelos de ML
from scipy.stats import mannwhitneyu # Librería para realizar pruebas de hipótesis
from sklearn.neighbors import KNeighborsClassifier #Para predecir los altos valores nulos
from scipy.stats import chi2_contingency

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
DATA_PATH = "/content/drive/MyDrive/NUCLIO/M7 - TFM/EXCEL_FILES/EXCEL_FILES_FILTERED"

## Carga del DATASET

### Dataset Unificado

In [ ]:
#Leemos todos los CSV
questionary_df = pd.read_csv('/content/drive/MyDrive/NUCLIO/M7 - TFM/EXCEL_FILES/EXCEL_FILES_FILTERED/QUESTIONARY.csv')
laboratory_df = pd.read_csv('/content/drive/MyDrive/NUCLIO/M7 - TFM/EXCEL_FILES/EXCEL_FILES_FILTERED/LABORATORY.csv')
demographics_df = pd.read_csv('/content/drive/MyDrive/NUCLIO/M7 - TFM/EXCEL_FILES/EXCEL_FILES_FILTERED/DEMOGRAPHIC.csv')
examination_df = pd.read_csv('/content/drive/MyDrive/NUCLIO/M7 - TFM/EXCEL_FILES/EXCEL_FILES_FILTERED/EXAMINATION.csv')

In [ ]:
demographics_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21904 entries, 0 to 21903
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   SEQN       21904 non-null  float64
 1   Gender     21904 non-null  float64
 2   Age        21904 non-null  float64
 3   Race       21904 non-null  float64
 4   Country    21885 non-null  float64
 5   Education  13513 non-null  float64
 6   Pregnancy  2791 non-null   float64
 7   Income     18811 non-null  float64
dtypes: float64(8)
memory usage: 1.3 MB


In [ ]:
# Esto crea un DataFrame con SEQN como índice y sin columnas extra
index_df = pd.DataFrame({'SEQN': questionary_df['SEQN'].unique()})

In [ ]:
index_df

SEQN
0      109263.0
1      109264.0
2      109265.0
3      109266.0
4      109267.0
...         ...
25526   93698.0
25527   93699.0
25528   93700.0
25529   93701.0
25530   93702.0

[25531 rows x 1 columns]

In [ ]:
seqn_base = set(index_df['SEQN'])
seqn_demo = set(demographics_df['SEQN'])
seqn_exam = set(examination_df['SEQN'])
seqn_labo = set(laboratory_df['SEQN'])
seqn_quest = set(questionary_df['SEQN'])

In [ ]:
print(f"SEQN totales en base: {len(seqn_base)}\n")

print(f"Coinciden con demographics_df: {len(seqn_base & seqn_demo)}")
print(f"Coinciden con examination_df: {len(seqn_base & seqn_exam)}")
print(f"Coinciden con laboratory_df: {len(seqn_base & seqn_labo)}")
print(f"Coinciden con questionary_df: {len(seqn_base & seqn_quest)}")


SEQN totales en base: 25531

Coinciden con demographics_df: 9971
Coinciden con examination_df: 23844
Coinciden con laboratory_df: 22937
Coinciden con questionary_df: 25531


Vamos a eliminar demographics, ya que el grupo poblacional es muy diferente a los otros 3 datasets

In [ ]:
# Asegurarte de que SEQN esté como columna (por si acaso)
for df in [examination_df, laboratory_df, questionary_df]:
    if 'SEQN' not in df.columns:
        df.reset_index(inplace=True)

# Crear conjuntos de SEQN únicos por cada dataset
seqn_exam = set(examination_df['SEQN'].dropna().astype(int))
seqn_labo = set(laboratory_df['SEQN'].dropna().astype(int))
seqn_quest = set(questionary_df['SEQN'].dropna().astype(int))

In [ ]:
# SEQN que están en los 3 datasets
seqn_comunes = seqn_exam & seqn_labo & seqn_quest
print(f"SEQN comunes en los 3 datasets: {len(seqn_comunes)}")

SEQN comunes en los 3 datasets: 22937


Nos quedamos con los 3 datasets, ya que coinciden en bastantes filas

In [ ]:
print(f"Solo en examination: {len(seqn_exam - seqn_comunes)}")
print(f"Solo en laboratory: {len(seqn_labo - seqn_comunes)}")
print(f"Solo en questionary: {len(seqn_quest - seqn_comunes)}")

Solo en examination: 907
Solo en laboratory: 0
Solo en questionary: 2594


In [ ]:
# SEQN que están en los 2 datasets
seqn_comunes_2 = seqn_exam & seqn_quest
print(f"SEQN comunes en los 2 datasets: {len(seqn_comunes)}")

SEQN comunes en los 2 datasets: 22937


In [ ]:
print(f"Solo en examination: {len(seqn_exam - seqn_comunes_2)}")
print(f"Solo en questionary: {len(seqn_quest - seqn_comunes_2)}")

Solo en examination: 0
Solo en questionary: 1687


In [ ]:
comunes_df = pd.DataFrame({'SEQN': list(seqn_comunes)})

In [ ]:
comunes_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22937 entries, 0 to 22936
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   SEQN    22937 non-null  int64
dtypes: int64(1)
memory usage: 179.3 KB


De momento vamos a analizar todos las filas, las 25531. Aunque quizas luego haremos ajustes

In [ ]:
# Fusionar uno por uno usando on="SEQN"
merged_df = pd.merge(examination_df, laboratory_df, on='SEQN', how='outer')
merged_df = pd.merge(merged_df, questionary_df, on='SEQN', how='outer')
merged_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25531 entries, 0 to 25530
Columns: 132 entries, SEQN to Ate_less_fastfood
dtypes: float64(132)
memory usage: 25.7 MB


In [ ]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25531 entries, 0 to 25530
Columns: 132 entries, SEQN to Ate_less_fastfood
dtypes: float64(132)
memory usage: 25.7 MB


In [ ]:
merged_df.head()

SEQN  Weight  Height   BMI  Upper_Leg_Length  Upper_Arm_Length  \
0  83732.0    94.8   184.5  27.8              43.3              43.6   
1  83733.0    90.4   171.4  30.8              38.0              40.0   
2  83734.0    83.4   170.1  28.8              35.6              37.0   
3  83735.0   109.8   160.9  42.4              38.5              37.7   
4  83736.0    55.2   164.9  20.3              37.4              36.0   

   Arm_Circumference  Waist_Circumference  Hip_Circumference  Systolic_mean  \
0               35.9                101.1               22.8     122.666667   
1               33.2                107.9               27.3     140.000000   
2               31.0                116.5               26.6     135.333333   
3               38.3                110.1               25.1     134.000000   
4               27.2                 80.4                NaN     104.000000   

   ...  Current_height  Current_weight  Weight_ 1yr_ago  Greatest_weight  \
0  ...            74.0           212.0            225.0            260.0   
1  ...            68.0           193.0            191.0            205.0   
2  ...            69.0           182.0            212.0            240.0   
3  ...            64.0           220.0            220.0            240.0   
4  ...            64.0           125.0            135.0            135.0   

   Age_heaviest_weight  Weight_change_intentional  Exercised_lose_weight  \
0                 20.0                        1.0                   13.0   
1                 51.0                        NaN                    NaN   
2                 46.0                        1.0                    NaN   
3                 55.0                        NaN                    NaN   
4                 33.0                        2.0                    NaN   

   Changed_eating_habits  Ate_less_sugar  Ate_less_fastfood  
0                    NaN             NaN               46.0  
1                    NaN             NaN                NaN  
2                    NaN             NaN                NaN  
3                    NaN             NaN               46.0  
4                    NaN             NaN                NaN  

[5 rows x 132 columns]

In [ ]:
#Guardar dataset demo en la carpeta EXCEL_FILE_FILTERED
merged_df.to_csv('/content/drive/MyDrive/NUCLIO/M7 - TFM/EXCEL_FILES/EXCEL_FILES_FILTERED/HEART_DISEASE_RAW.csv', index=False)

In [ ]:
print(os.listdir(DATA_PATH))

['EXAMINATION.csv', 'LABORATORY.csv', 'QUESTIONARY.csv', 'DEMOGRAPHIC.csv', 'DATAFRAME_FILAS.csv', 'HEART_DISEASE_RAW.csv', 'HEART_DISEASE_TARGET.csv', 'HEART_DISEASE_NULLS.csv']


### DataLoader

In [ ]:
'''
El DataLoader es una clase auxiliar que va a cargar nuestros datasets
Le suministramos la ruta del train/X y del test/dataset producción (prod) y nos devuelve los dos datasets
También le tenemos que pasar la columna del target y del index
Para que el objeto DataLoader distinga estas dos columnas
'''

class DataLoader(object):
    '''
    DataLoader helps you import you train and test data and do some basic preprocessing on them.
    '''
    def __init__(self, train_path, train_columns, target, index):
        '''
        Constructor for the class.
        Needs the train and test path and train_columns (features), index and target column.
        '''
        self.train_path = train_path

        self.train_columns = train_columns
        self.target = target
        self.index = index

    def _process_df(self, df):
        '''
        Converts the columns to upper, sets index and splits between X and y.
        '''

        df.columns = map(str.upper, df.columns)


        if self.target.upper() not in df.columns:
            raise KeyError(f"Target column '{self.target}' not found in the DataFrame.")
        if self.index.upper() not in df.columns:
            raise KeyError(f"Index column '{self.index}' not found in the DataFrame.")

        df.set_index(self.index.upper(), inplace=True)

        if self.target.upper() in df.columns:

            y = df[self.target.upper()]
            df.drop(self.target.upper(), axis=1, inplace=True)


            train_columns_upper = [col.upper() for col in self.train_columns]


            missing_cols = set(train_columns_upper) - set(df.columns)
            if missing_cols:
                raise KeyError(f"The following columns are missing in the DataFrame: {list(missing_cols)}")

            df = df[train_columns_upper]

            return df, y

        else:

            train_columns_upper = [col.upper() for col in self.train_columns]
            df = df[train_columns_upper]

            return df, None

    def load_data(self):
        '''
        Loads the data and calls _process_df to X_train and X_test.
        '''
        X_train, y_train = self._process_df(pd.read_csv(self.train_path))

        return X_train, y_train

### Dataframe reporter

In [ ]:
'''
DataFrameReporter nos va a permitir hacer nuestro primer contacto con el dataset.

Para las variables númericas hará un describe y contará los nulos que hay.
Y para las variables categóricas hará un count de los nulos y nos dirá también
el número de categorías únicas que hay.

Al final se trata de una clase auxiliar que hará una parte del EDA.
'''

class DataFrameReporter:
    '''
    Generates a unified report for each column in the DataFrame,
    including nulls, stats, and unique category count.
    '''
    def __init__(self, X, target_column=None):
        self.X = X.drop(columns=[target_column], errors='ignore') if target_column else X

    def generate_report(self):
        report_rows = []

        for col in self.X.columns:
            series = self.X[col]
            dtype = series.dtype
            shape = series.shape[0]
            abs_nulls = series.isnull().sum()
            rel_nulls = abs_nulls / shape
            count = series.count()
            unique_vals = series.nunique()

            # Numéricos
            if pd.api.types.is_numeric_dtype(series):
                stats = series.describe(percentiles=[.25, .5, .75])
                row = {
                    "Feature": col,
                    "Dtype": dtype,
                    "Shape": shape,
                    "Absolute_nulls": abs_nulls,
                    "Relative_nulls": rel_nulls,
                    "Count": count,
                    "Mean": stats.get("mean", np.nan),
                    "Std": stats.get("std", np.nan),
                    "Min": stats.get("min", np.nan),
                    "25%": stats.get("25%", np.nan),
                    "50%": stats.get("50%", np.nan),
                    "75%": stats.get("75%", np.nan),
                    "Max": stats.get("max", np.nan),
                    "Unique_category": unique_vals
                }
            else:
                # Categóricas o booleanas
                row = {
                    "Feature": col,
                    "Dtype": dtype,
                    "Shape": shape,
                    "Absolute_nulls": abs_nulls,
                    "Relative_nulls": rel_nulls,
                    "Count": count,
                    "Mean": np.nan,
                    "Std": np.nan,
                    "Min": np.nan,
                    "25%": np.nan,
                    "50%": np.nan,
                    "75%": np.nan,
                    "Max": np.nan,
                    "Unique_category": unique_vals
                }

            report_rows.append(row)

        return pd.DataFrame(report_rows)

### Creación del Report Inicial

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/NUCLIO/M7 - TFM/EXCEL_FILES/EXCEL_FILES_FILTERED/HEART_DISEASE_RAW.csv')

In [ ]:
print(df.columns.tolist())

['SEQN', 'Weight', 'Height', 'BMI', 'Upper_Leg_Length', 'Upper_Arm_Length', 'Arm_Circumference', 'Waist_Circumference', 'Hip_Circumference', 'Systolic_mean', 'Diastolic_mean', 'Pulse_mean', 'Albumin_urine', 'Creatinine_urine', 'Album_creat_ratio', 'HDL', 'Triglycerides', 'LDL', 'Total_Chole', 'White_blood_cell', 'Lymphocyte', 'Monocyte', 'Segm_neutrophils', 'Eosinophils', 'Basophils', 'Red_blood_cell', 'Hemoglobin', 'Mean_cell_vol', 'Red_cell_dist', 'Platelet', 'Mean_platelet_vol', 'Cotinine', 'Hydroxycotinine', 'Ferritin', 'Folate', 'Serum_total_folate', 'Methyl_tetrahydrofolate', 'Folic_acid', 'Formyl_tetrahydrofolate', 'Tetrahydrofolate', 'Methenyl_tetrahydrofolate', 'Mefox_oxidation', 'Glycohemoglobin', 'HepA_antibody', 'HepBS_antibody', 'HepBC_antibody', 'HepBS_antigen', 'HepD_antibody', 'HepC_RNA', 'HepC_antibody', 'HepC_genotype', 'HepE_IgG', 'HepE_IgM', 'CReactiveProtein_HS', 'Blood_InorganicMercury', 'EthylMercury_Blood', 'MethylMercury_Blood', 'Insulin', 'Plomo_sangre', 'Cadm

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25531 entries, 0 to 25530
Columns: 132 entries, SEQN to Ate_less_fastfood
dtypes: float64(132)
memory usage: 25.7 MB


In [ ]:
# Definimos nuestras variables globales
# En este caso, vamos a usar unas pocas columnas del dataframe a modo de ejemplo
# Es buena praxis definir este tipo de variables en un fichero json o bien al comienzo de un
# notebook/script

TRAIN_PATH = os.path.join(DATA_PATH, "HEART_DISEASE_RAW.csv")

TARGET = "coronary_heart_disease"
INDEX = "SEQN"


TRAIN_COLUMNS = [i for i in df if i != TARGET and i != INDEX]


# Cargamos nuestros datasets con la clase auxiliar

X, y = DataLoader(
    train_path = TRAIN_PATH,
    train_columns = TRAIN_COLUMNS,
    target = TARGET,
    index = INDEX
).load_data()

In [ ]:
# Hacemos nuestro análisis rápido de los datasets
report = DataFrameReporter(
    pd.concat([X, y], axis = 1), TARGET
).generate_report()

In [ ]:
report

Feature    Dtype  Shape  Absolute_nulls  Relative_nulls  \
0                    WEIGHT  float64  25531            2011        0.078767   
1                    HEIGHT  float64  25531            3605        0.141201   
2                       BMI  float64  25531            3638        0.142493   
3          UPPER_LEG_LENGTH  float64  25531            7437        0.291293   
4          UPPER_ARM_LENGTH  float64  25531            3065        0.120050   
..                      ...      ...    ...             ...             ...   
126   EXERCISED_LOSE_WEIGHT  float64  25531           20680        0.809996   
127   CHANGED_EATING_HABITS  float64  25531           22331        0.874662   
128          ATE_LESS_SUGAR  float64  25531           22195        0.869335   
129       ATE_LESS_FASTFOOD  float64  25531           21887        0.857272   
130  CORONARY_HEART_DISEASE  float64  25531           10580        0.414398   

     Count        Mean        Std   Min    25%    50%    75%    Max  \
0    23520   64.237759  32.978013   3.2   39.9   67.0   85.3  254.3   
1    21926  155.926498  22.784454  78.3  150.1  161.7  170.9  202.7   
2    21893   26.400822   8.246289  11.5   20.2   25.6   31.1   92.3   
3    18094   38.488764   4.157066  24.0   35.7   38.6   41.3   55.0   
4    22466   33.487096   7.374410   9.4   31.1   35.8   38.4   49.9   
..     ...         ...        ...   ...    ...    ...    ...    ...   
126   4851   13.000000   0.000000  13.0   13.0   13.0   13.0   13.0   
127   3200   44.000000   0.000000  44.0   44.0   44.0   44.0   44.0   
128   3336   45.000000   0.000000  45.0   45.0   45.0   45.0   45.0   
129   3644   46.000000   0.000000  46.0   46.0   46.0   46.0   46.0   
130  14951    1.980670   0.470353   1.0    2.0    2.0    2.0    9.0   

     Unique_category  
0               1569  
1               1137  
2                499  
3                267  
4                373  
..               ...  
126                1  
127                1  
128                1  
129                1  
130                3  

[131 rows x 14 columns]

In [ ]:
df2 = df.copy()

## EDA INICIAL

### Limpieza Columnas

In [ ]:
#Eliminar las columnas que tengan Relative_nulls > 0.60
columns_to_drop = report[report["Relative_nulls"] > 0.50]["Feature"].tolist()
columns_to_drop

['TRIGLYCERIDES',
 'LDL',
 'HEPBS_ANTIGEN',
 'HEPD_ANTIBODY',
 'HEPC_ANTIBODY',
 'HEPC_GENOTYPE',
 'INSULIN',
 'CADMIUM_URINE',
 'LEAD_URINE',
 'THALLIUM_URINE',
 'DRINKS/DAY(1Y)',
 '4-5DRINKS/DAY',
 'DR_MEDICINE_HBP',
 'TAKING_MEDICINE_HBP ',
 'DR_MEDICINE_CHOLE',
 'TAKING_MEDICINE_CHOLE',
 'EVER_PAIN_CHEST',
 'SHORTNESS_BREATH',
 'RECENT_DIASTOLIC_BP',
 'TARGET_SYSTOLIC_BP',
 'TARGET_DIASTOLIC_BP',
 'TARGET_LDL',
 'AGE_STARTED',
 'SMOKE_CIGARETTES',
 'TIME-QUIT_SMOKING',
 'AVG_DURING30DAYS',
 'WEIGHT_CHANGE_INTENTIONAL',
 'EXERCISED_LOSE_WEIGHT',
 'CHANGED_EATING_HABITS',
 'ATE_LESS_SUGAR',
 'ATE_LESS_FASTFOOD']

In [ ]:
len(columns_to_drop)


31

In [ ]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25531 entries, 0 to 25530
Columns: 132 entries, SEQN to Ate_less_fastfood
dtypes: float64(132)
memory usage: 25.7 MB


In [ ]:
#Eliminamos columnas con mas de 75% nulos
df2.columns = [col.upper() for col in df2.columns]
df2 = df2.drop(columns=columns_to_drop, errors='ignore')

In [ ]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25531 entries, 0 to 25530
Columns: 101 entries, SEQN to AGE_HEAVIEST_WEIGHT
dtypes: float64(101)
memory usage: 19.7 MB


In [ ]:
# Definimos nuestras variables globales
# En este caso, vamos a usar unas pocas columnas del dataframe a modo de ejemplo
# Es buena praxis definir este tipo de variables en un fichero json o bien al comienzo de un
# notebook/script

TRAIN_PATH = os.path.join(DATA_PATH, "HEART_DISEASE_RAW.csv")

TARGET = "UPPER_LEG_LENGTH"
INDEX = "SEQN"


TRAIN_COLUMNS = [i for i in df2 if i != TARGET and i != INDEX]


# Cargamos nuestros datasets con la clase auxiliar

X, y = DataLoader(
    train_path = TRAIN_PATH,
    train_columns = TRAIN_COLUMNS,
    target = TARGET,
    index = INDEX
).load_data()

# Hacemos nuestro análisis rápido de los datasets
report = DataFrameReporter(
    pd.concat([X, y], axis = 1), TARGET
).generate_report()

report

Feature    Dtype  Shape  Absolute_nulls  Relative_nulls  \
0                WEIGHT  float64  25531            2011        0.078767   
1                HEIGHT  float64  25531            3605        0.141201   
2                   BMI  float64  25531            3638        0.142493   
3      UPPER_ARM_LENGTH  float64  25531            3065        0.120050   
4     ARM_CIRCUMFERENCE  float64  25531            3071        0.120285   
..                  ...      ...    ...             ...             ...   
94       CURRENT_HEIGHT  float64  25531            9066        0.355098   
95       CURRENT_WEIGHT  float64  25531            9009        0.352865   
96      WEIGHT_ 1YR_AGO  float64  25531            9009        0.352865   
97      GREATEST_WEIGHT  float64  25531            9847        0.385688   
98  AGE_HEAVIEST_WEIGHT  float64  25531           10109        0.395950   

    Count        Mean          Std   Min    25%    50%    75%      Max  \
0   23520   64.237759    32.978013   3.2   39.9   67.0   85.3    254.3   
1   21926  155.926498    22.784454  78.3  150.1  161.7  170.9    202.7   
2   21893   26.400822     8.246289  11.5   20.2   25.6   31.1     92.3   
3   22466   33.487096     7.374410   9.4   31.1   35.8   38.4     49.9   
4   22460   28.997373     8.080053  10.1   23.2   30.0   34.6     64.5   
..    ...         ...          ...   ...    ...    ...    ...      ...   
94  16465  280.772487  1443.100973  47.0   63.0   66.0   70.0   9999.0   
95  16522  354.444196  1294.943644  67.0  145.0  172.0  206.0   9999.0   
96  16522  378.459145  1381.129463  60.0  145.0  172.0  210.0   9999.0   
97  15684  353.834098  1219.271584  75.0  160.0  190.0  230.0   9999.0   
98  15422  800.975490  8666.560470  10.0   26.0   39.0   54.0  99999.0   

    Unique_category  
0              1569  
1              1137  
2               499  
3               373  
4               426  
..              ...  
94               40  
95              306  
96              308  
97              352  
98               73  

[99 rows x 14 columns]

### Comprobacion Target

In [ ]:
columns_to_check = [
    'CONGESTIVE_HEART_FAILURE',
    'CORONARY_HEART_DISEASE',
    'ANGINA_PECTORIS',
    'HEART_ATTACK',
    'HAD_A_STROKE',
    'CLOSE_HEART_ATTACK'
]

positive_rows_mask = (df2[columns_to_check] == 1.0).any(axis=1)
df2_positive = df2[positive_rows_mask]
df2_positive[columns_to_check]

CONGESTIVE_HEART_FAILURE  CORONARY_HEART_DISEASE  ANGINA_PECTORIS  \
2                           2.0                     2.0              2.0   
9                           2.0                     2.0              2.0   
12                          1.0                     2.0              2.0   
23                          2.0                     2.0              1.0   
25                          1.0                     1.0              2.0   
...                         ...                     ...              ...   
25468                       2.0                     2.0              2.0   
25485                       2.0                     2.0              2.0   
25502                       2.0                     2.0              2.0   
25522                       2.0                     2.0              2.0   
25530                       2.0                     2.0              2.0   

       HEART_ATTACK  HAD_A_STROKE  CLOSE_HEART_ATTACK  
2               1.0           2.0                 2.0  
9               2.0           2.0                 1.0  
12              1.0           2.0                 2.0  
23              2.0           2.0                 2.0  
25              2.0           1.0                 1.0  
...             ...           ...                 ...  
25468           2.0           2.0                 1.0  
25485           2.0           2.0                 1.0  
25502           2.0           1.0                 2.0  
25522           2.0           1.0                 1.0  
25530           1.0           2.0                 2.0  

[3221 rows x 6 columns]

In [ ]:
total_positive_rows = positive_rows_mask.sum()
print(f"Total de filas con al menos un valor positivo en esas columnas: {total_positive_rows}")

Total de filas con al menos un valor positivo en esas columnas: 3221


In [ ]:
df2.shape

(25531, 101)

In [ ]:
df2['HEART_DISEASE'] = np.where((df2[columns_to_check] == 1.0).any(axis=1), 1,
                               np.where((df2[columns_to_check].isnull()).any(axis=1), np.nan, 0))


In [ ]:
df2['HEART_DISEASE'].head(10)

0    0.0
1    0.0
2    1.0
3    0.0
4    0.0
5    0.0
6    NaN
7    NaN
8    NaN
9    1.0
Name: HEART_DISEASE, dtype: float64

In [ ]:
heart_disease_count = (df2['HEART_DISEASE'] == 1.0).sum()
print(f"Number of rows where 'HEART_DISEASE' is 1.0: {heart_disease_count}")

Number of rows where 'HEART_DISEASE' is 1.0: 3221


In [ ]:
df2 = df2.drop(columns=columns_to_check)

In [ ]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25531 entries, 0 to 25530
Data columns (total 96 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   SEQN                           25531 non-null  float64
 1   WEIGHT                         23520 non-null  float64
 2   HEIGHT                         21926 non-null  float64
 3   BMI                            21893 non-null  float64
 4   UPPER_LEG_LENGTH               18094 non-null  float64
 5   UPPER_ARM_LENGTH               22466 non-null  float64
 6   ARM_CIRCUMFERENCE              22460 non-null  float64
 7   WAIST_CIRCUMFERENCE            20887 non-null  float64
 8   HIP_CIRCUMFERENCE              16845 non-null  float64
 9   SYSTOLIC_MEAN                  17716 non-null  float64
 10  DIASTOLIC_MEAN                 17716 non-null  float64
 11  PULSE_MEAN                     16848 non-null  float64
 12  ALBUMIN_URINE                  20790 non-null 

In [ ]:
df2["HEART_DISEASE"]

0        0.0
1        0.0
2        1.0
3        0.0
4        0.0
        ... 
25526    0.0
25527    NaN
25528    NaN
25529    0.0
25530    1.0
Name: HEART_DISEASE, Length: 25531, dtype: float64

In [ ]:
# Guardar el DataFrame actualizado
df2.to_csv('/content/drive/MyDrive/NUCLIO/M7 - TFM/EXCEL_FILES/EXCEL_FILES_FILTERED/HEART_DISEASE_TARGET.csv', index=False)

In [ ]:
# Definimos nuestras variables globales
# En este caso, vamos a usar unas pocas columnas del dataframe a modo de ejemplo
# Es buena praxis definir este tipo de variables en un fichero json o bien al comienzo de un
# notebook/script

TRAIN_PATH = os.path.join(DATA_PATH, "HEART_DISEASE_TARGET.csv")

TARGET = "FOLIC_ACID"
INDEX = "SEQN"


TRAIN_COLUMNS = [i for i in df2 if i != TARGET and i != INDEX]


# Cargamos nuestros datasets con la clase auxiliar

X, y = DataLoader(
    train_path = TRAIN_PATH,
    train_columns = TRAIN_COLUMNS,
    target = TARGET,
    index = INDEX
).load_data()

In [ ]:
# Hacemos nuestro análisis rápido de los datasets
report = DataFrameReporter(
    pd.concat([X, y], axis = 1), TARGET
).generate_report()
report

Feature    Dtype  Shape  Absolute_nulls  Relative_nulls  \
0                WEIGHT  float64  25531            2011        0.078767   
1                HEIGHT  float64  25531            3605        0.141201   
2                   BMI  float64  25531            3638        0.142493   
3      UPPER_LEG_LENGTH  float64  25531            7437        0.291293   
4      UPPER_ARM_LENGTH  float64  25531            3065        0.120050   
..                  ...      ...    ...             ...             ...   
89       CURRENT_WEIGHT  float64  25531            9009        0.352865   
90      WEIGHT_ 1YR_AGO  float64  25531            9009        0.352865   
91      GREATEST_WEIGHT  float64  25531            9847        0.385688   
92  AGE_HEAVIEST_WEIGHT  float64  25531           10109        0.395950   
93        HEART_DISEASE  float64  25531           10580        0.414398   

    Count        Mean          Std   Min    25%    50%    75%      Max  \
0   23520   64.237759    32.978013   3.2   39.9   67.0   85.3    254.3   
1   21926  155.926498    22.784454  78.3  150.1  161.7  170.9    202.7   
2   21893   26.400822     8.246289  11.5   20.2   25.6   31.1     92.3   
3   18094   38.488764     4.157066  24.0   35.7   38.6   41.3     55.0   
4   22466   33.487096     7.374410   9.4   31.1   35.8   38.4     49.9   
..    ...         ...          ...   ...    ...    ...    ...      ...   
89  16522  354.444196  1294.943644  67.0  145.0  172.0  206.0   9999.0   
90  16522  378.459145  1381.129463  60.0  145.0  172.0  210.0   9999.0   
91  15684  353.834098  1219.271584  75.0  160.0  190.0  230.0   9999.0   
92  15422  800.975490  8666.560470  10.0   26.0   39.0   54.0  99999.0   
93  14951    0.215437     0.411139   0.0    0.0    0.0    0.0      1.0   

    Unique_category  
0              1569  
1              1137  
2               499  
3               267  
4               373  
..              ...  
89              306  
90              308  
91              352  
92               73  
93                2  

[94 rows x 14 columns]

Para HEART_DISEASE tenemos 10580 nulos. Vamos a guardar esas filas para la prediccion del TARGET una vez desarrollemos el modelo.

In [ ]:
null_heart_disease_rows = df2[df2['HEART_DISEASE'].isnull()]
null_heart_disease_rows
null_heart_disease_rows.to_csv('/content/drive/MyDrive/NUCLIO/M7 - TFM/EXCEL_FILES/EXCEL_FILES_FILTERED/HEART_DISEASE_NULLS.csv', index=False)



In [ ]:
df2 = df2.dropna(subset=['HEART_DISEASE'])
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14951 entries, 0 to 25530
Data columns (total 96 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   SEQN                           14951 non-null  float64
 1   WEIGHT                         13812 non-null  float64
 2   HEIGHT                         13812 non-null  float64
 3   BMI                            13787 non-null  float64
 4   UPPER_LEG_LENGTH               13106 non-null  float64
 5   UPPER_ARM_LENGTH               13322 non-null  float64
 6   ARM_CIRCUMFERENCE              13321 non-null  float64
 7   WAIST_CIRCUMFERENCE            13171 non-null  float64
 8   HIP_CIRCUMFERENCE              13036 non-null  float64
 9   SYSTOLIC_MEAN                  12963 non-null  float64
 10  DIASTOLIC_MEAN                 12963 non-null  float64
 11  PULSE_MEAN                     12303 non-null  float64
 12  ALBUMIN_URINE                  13667 non-null  floa

In [ ]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14951 entries, 0 to 25530
Data columns (total 96 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   SEQN                           14951 non-null  float64
 1   WEIGHT                         13812 non-null  float64
 2   HEIGHT                         13812 non-null  float64
 3   BMI                            13787 non-null  float64
 4   UPPER_LEG_LENGTH               13106 non-null  float64
 5   UPPER_ARM_LENGTH               13322 non-null  float64
 6   ARM_CIRCUMFERENCE              13321 non-null  float64
 7   WAIST_CIRCUMFERENCE            13171 non-null  float64
 8   HIP_CIRCUMFERENCE              13036 non-null  float64
 9   SYSTOLIC_MEAN                  12963 non-null  float64
 10  DIASTOLIC_MEAN                 12963 non-null  float64
 11  PULSE_MEAN                     12303 non-null  float64
 12  ALBUMIN_URINE                  13667 non-null  floa

In [ ]:
# Guardar el DataFrame actualizado
df2.to_csv('/content/drive/MyDrive/NUCLIO/M7 - TFM/EXCEL_FILES/EXCEL_FILES_FILTERED/HEART_DISEASE_TARGET.csv', index=False)

### Limpieza de Filas

In [ ]:
df2.shape

(14951, 96)

In [ ]:
#Eliminar las filas que tengan más del 50% de nulos en total de todas las columnas
filas_to_drop = df2[df2.isnull().sum(axis=1) / len(df2.columns) > 0.50].index
filas_to_drop

Index([   26,    54,    92,   114,   129,   144,   161,   172,   175,   195,
       ...
       25380, 25395, 25408, 25411, 25446, 25459, 25468, 25491, 25513, 25516],
      dtype='int64', length=1643)

In [ ]:
df2.drop(filas_to_drop, axis=0, inplace=True)

In [ ]:
df2.shape

(13308, 96)

In [ ]:
#Creamos una copia
df2.to_csv('/content/drive/MyDrive/NUCLIO/M7 - TFM/EXCEL_FILES/EXCEL_FILES_FILTERED/HEART_DISEASE_TARGET.csv', index=False)

### Creación Report V2

In [ ]:
# Definimos nuestras variables globales
# En este caso, vamos a usar unas pocas columnas del dataframe a modo de ejemplo
# Es buena praxis definir este tipo de variables en un fichero json o bien al comienzo de un
# notebook/script

TRAIN_PATH = os.path.join(DATA_PATH, "HEART_DISEASE_TARGET.csv")

TARGET = "HEART_DISEASE"
INDEX = "SEQN"


TRAIN_COLUMNS = [i for i in df2 if i != TARGET and i != INDEX]


# Cargamos nuestros datasets con la clase auxiliar

X, y = DataLoader(
    train_path = TRAIN_PATH,
    train_columns = TRAIN_COLUMNS,
    target = TARGET,
    index = INDEX
).load_data()

In [ ]:
# Inicializamos listas
numeric_columns = []
categorical_columns = []
boolean_columns = []

for col in X.columns:
    unique_vals = X[col].dropna().unique()
    num_unique = len(unique_vals)

    # Booleanas por contenido: solo 2 valores únicos (como True/False, 0/1, 'yes'/'no', etc.)
    if num_unique <= 4:
        boolean_columns.append(col)

    # Numéricas: dtype numérico y más de 2 valores
    elif pd.api.types.is_numeric_dtype(X[col]):
        numeric_columns.append(col)

    # Categóricas: tipo object, string, o pocas categorías aunque sea numérica
    else:
        categorical_columns.append(col)

# Eliminamos booleanas de las otras listas si se colaron
numeric_columns = [col for col in numeric_columns if col not in boolean_columns]
categorical_columns = [col for col in categorical_columns if col not in boolean_columns]

# Mostramos resultados
print("Numéricas:", numeric_columns)
print("Categoricas:", categorical_columns)
print("Booleanas (por contenido):", boolean_columns)

# Validación
total_detected = len(numeric_columns) + len(categorical_columns) + len(boolean_columns)
assert total_detected == X.shape[1], f"Columnas sin clasificar: {X.shape[1] - total_detected}"


Numéricas: ['WEIGHT', 'HEIGHT', 'BMI', 'UPPER_LEG_LENGTH', 'UPPER_ARM_LENGTH', 'ARM_CIRCUMFERENCE', 'WAIST_CIRCUMFERENCE', 'HIP_CIRCUMFERENCE', 'SYSTOLIC_MEAN', 'DIASTOLIC_MEAN', 'PULSE_MEAN', 'ALBUMIN_URINE', 'CREATININE_URINE', 'ALBUM_CREAT_RATIO', 'HDL', 'TOTAL_CHOLE', 'WHITE_BLOOD_CELL', 'LYMPHOCYTE', 'MONOCYTE', 'SEGM_NEUTROPHILS', 'EOSINOPHILS', 'BASOPHILS', 'RED_BLOOD_CELL', 'HEMOGLOBIN', 'MEAN_CELL_VOL', 'RED_CELL_DIST', 'PLATELET', 'MEAN_PLATELET_VOL', 'COTININE', 'HYDROXYCOTININE', 'FERRITIN', 'FOLATE', 'SERUM_TOTAL_FOLATE', 'METHYL_TETRAHYDROFOLATE', 'FOLIC_ACID', 'FORMYL_TETRAHYDROFOLATE', 'TETRAHYDROFOLATE', 'METHENYL_TETRAHYDROFOLATE', 'MEFOX_OXIDATION', 'GLYCOHEMOGLOBIN', 'CREACTIVEPROTEIN_HS', 'BLOOD_INORGANICMERCURY', 'ETHYLMERCURY_BLOOD', 'METHYLMERCURY_BLOOD', 'PLOMO_SANGRE', 'CADMIO_SANGRE', 'MERCURIO_SANGRE', 'SELENIO_SANGRE', 'MANGANESIO_SANGRE', 'ALBUMIN_BLOOD', 'ALKALINE_PHOSPHATASE', 'ALT_GPT', 'AST_GOT', 'BLOOD_UREA_NITROGEN', 'COMPLEMENT_C3', 'CALCIUM', 'CREA

In [ ]:
# Hacemos nuestro análisis rápido de los datasets
report = DataFrameReporter(
    pd.concat([X, y], axis = 1), TARGET
).generate_report()

In [ ]:
report

Feature    Dtype  Shape  Absolute_nulls  Relative_nulls  \
0                WEIGHT  float64  13308             160        0.012023   
1                HEIGHT  float64  13308             165        0.012399   
2                   BMI  float64  13308             184        0.013826   
3      UPPER_LEG_LENGTH  float64  13308             752        0.056507   
4      UPPER_ARM_LENGTH  float64  13308             545        0.040953   
..                  ...      ...    ...             ...             ...   
89       CURRENT_HEIGHT  float64  13308              52        0.003907   
90       CURRENT_WEIGHT  float64  13308               0        0.000000   
91      WEIGHT_ 1YR_AGO  float64  13308               0        0.000000   
92      GREATEST_WEIGHT  float64  13308               1        0.000075   
93  AGE_HEAVIEST_WEIGHT  float64  13308             188        0.014127   

    Count        Mean          Std    Min    25%    50%    75%      Max  \
0   13148   82.962458    22.496742   32.4   67.3   79.5   94.8    254.3   
1   13143  166.412866    10.077389  129.7  158.9  166.1  173.7    202.7   
2   13124   29.865064     7.337105   14.2   24.8   28.7   33.5     92.3   
3   12556   38.852533     3.861008   25.7   36.2   38.9   41.5     53.0   
4   12763   37.437867     2.857870   28.2   35.5   37.4   39.4     49.9   
..    ...         ...          ...    ...    ...    ...    ...      ...   
89  13256  286.213111  1460.826142   48.0   63.0   66.0   70.0   9999.0   
90  13308  338.108882  1222.380658   75.0  149.0  175.0  210.0   9999.0   
91  13308  351.014578  1270.793518   60.0  148.0  175.0  212.0   9999.0   
92  13307  329.961599  1114.278244   75.0  160.0  190.0  230.0   9999.0   
93  13120  718.112805  8186.838309   11.0   28.0   40.0   55.0  99999.0   

    Unique_category  
0              1186  
1               553  
2               466  
3               244  
4               183  
..              ...  
89               39  
90              293  
91              295  
92              338  
93               72  

[94 rows x 14 columns]